In [1]:
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeeds

import pandas as pd
import datetime

data = pd.read_csv('AAPL.csv')

data.set_index('Date', inplace = True)
data.columns = [i.lower() for i in data.columns]
data['close'] = data['adj close']
data = data.drop(['adj close'], axis = 1)
data['openinterest'] = 0
data.index = pd.to_datetime(data.index)
start_date = data.index[0]
end_date = data.index[-1]

# 实例化 cerebro
cerebro = bt.Cerebro()
datafeed = bt.feeds.PandasData(dataname = data, fromdate = start_date, todate = end_date)
cerebro.adddata(datafeed, name = 'AAPL')

# 通过经纪商设置初始资金
cerebro.broker.setcash(1000000)
# 设置交易佣金
cerebro.broker.setcommission(commission=0.0003)
cerebro.broker.set_slippage_perc(perc = 0.0001)

class TestStrategy(bt.Strategy):
    # 可选，设置回测的可变参数：如移动均线的周期
    params = (
        ('long',20),
        ('short', 5), # 最后一个“,”最好别删！
    )
    def log(self, txt, dt=None):
        '''可选，构建策略打印日志的函数：可用于打印订单记录或交易记录等'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        '''必选，初始化属性、计算指标等'''
        self.order = None
        self.dataclose = self.datas[0].close
        
        self.sma_20 = bt.indicators.SMA(self.datas[0], period = 20)
        self.sma_5 = bt.indicators.SMA(self.datas[0], period = 5)


        pass

    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已经处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                        'BUY EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                        (order.ref, # 订单编号
                        order.executed.price, # 成交价
                        order.executed.value, # 成交额
                        order.executed.comm, # 佣金
                        order.executed.size, # 成交量
                        order.data._name)) # 股票名称
            else: # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                            (order.ref,
                            order.executed.price,
                            order.executed.value,
                            order.executed.comm,
                            order.executed.size,
                            order.data._name))
        pass


    def notify_trade(self, trade):
        '''可选，打印交易信息'''
        pass

    def next(self):
        '''必选，编写交易策略逻辑'''

        if self.sma_5 > self.sma_20:
            self.log('BUY CREATE, %.2f' % self.dataclose[0])
            self.buy()
        
        if self.sma_5 < self.sma_20:
            self.log('SELL CREATE, %.2f' % self.dataclose[0])
            self.close()
        
        pass

# 添加策略
cerebro.addstrategy(TestStrategy)
# 添加策略分析指标
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # 回撤
# 启动回测
result = cerebro.run()
strat = result[0]
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())

# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())

# 可视化回测结果
cerebro.plot()


1981-01-12, BUY CREATE, 0.11
1981-01-13, BUY EXECUTED, ref:1, Price: 0.14, Cost: 0.14, Comm 0.00, Size: 1.00, Stock: AAPL
1981-01-13, SELL CREATE, 0.11
1981-01-14, SELL EXECUTED, ref:2, Price: 0.14, Cost: 0.14, Comm 0.00, Size: -1.00, Stock: AAPL
1981-01-14, SELL CREATE, 0.11
1981-01-15, SELL CREATE, 0.11
1981-01-16, SELL CREATE, 0.11
1981-01-19, SELL CREATE, 0.11
1981-01-20, SELL CREATE, 0.11
1981-01-21, SELL CREATE, 0.11
1981-01-22, SELL CREATE, 0.11
1981-01-23, SELL CREATE, 0.11
1981-01-26, BUY CREATE, 0.11
1981-01-27, BUY EXECUTED, ref:3, Price: 0.14, Cost: 0.14, Comm 0.00, Size: 1.00, Stock: AAPL
1981-01-27, BUY CREATE, 0.11
1981-01-28, BUY EXECUTED, ref:4, Price: 0.14, Cost: 0.14, Comm 0.00, Size: 1.00, Stock: AAPL
1981-01-28, BUY CREATE, 0.11
1981-01-29, BUY EXECUTED, ref:5, Price: 0.13, Cost: 0.13, Comm 0.00, Size: 1.00, Stock: AAPL
1981-01-29, SELL CREATE, 0.10
1981-01-30, SELL EXECUTED, ref:6, Price: 0.13, Cost: 0.42, Comm 0.00, Size: -3.00, Stock: AAPL
1981-01-30, SELL CREAT

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]

In [3]:
cerebro.plot(iplot = False)

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]

In [42]:
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeeds

import pandas as pd
import datetime

data = pd.read_csv('AAPL.csv')

data.set_index('Date', inplace = True)
data.columns = [i.lower() for i in data.columns]
data['close'] = data['adj close']
data = data.drop(['adj close'], axis = 1)
data['openinterest'] = 0
data.index = pd.to_datetime(data.index)
data = data.loc[datetime.datetime(2000,1,1):,:]
start_date = data.index[0]
end_date = data.index[-1]

class PandasData_more(bt.feeds.PandasData):
    lines = ('pe', 'pb', ) # 要添加的线
    # 设置 line 在数据源上的列位置
    params=(
        ('pe', -1),
        ('pb', -1),
           )

# 实例化 cerebro
cerebro = bt.Cerebro()
datafeed = PandasData_more(dataname = data, fromdate = start_date, todate = end_date)
cerebro.adddata(datafeed, name = 'AAPL')

# 通过经纪商设置初始资金
cerebro.broker.setcash(10000000)
# 设置交易佣金
cerebro.broker.setcommission(commission=0.0003)
cerebro.broker.set_slippage_perc(perc = 0.0001)
cerebro.broker.set_filler(bt.broker.fillers.BarPointPerc(minmov = 0.01, perc = 0.5))

class strategy(bt.Strategy):
    def __init__(self):
        # 打印数据集和数据集对应的名称
        print("-------------self.datas-------------")
        print(self.datas)
        print("-------------self.data-------------")
        print(self.data._name, self.data) # 返回第一个导入的数据表格，缩写形式
        print("-------------self.data0-------------")
        print(self.data0._name, self.data0) # 返回第一个导入的数据表格，缩写形式
        print("-------------self.datas[0]-------------")
        print(self.datas[0]._name, self.datas[0]) # 返回第一个导入的数据表格，常规形式
        sma = bt.indicators.SMA(self.datas[0].close)
        print(sma.lines[0])

        print("0 索引：",'datetime',self.data0.lines.datetime.date(0), 'close',self.data0.lines.close[0])
        print("-1 索引：",'datetime',self.data0.lines.datetime.date(-1),'close', self.data0.lines.close[-1])
        print("-2 索引",'datetime', self.data0.lines.datetime.date(-2),'close', self.data0.lines.close[-2])
        print("1 索引：",'datetime',self.data0.lines.datetime.date(1),'close', self.data0.lines.close[1])
        print("2 索引",'datetime', self.data0.lines.datetime.date(2),'close', self.data0.lines.close[2])
        print("从 0 开始往前取3天的收盘价:", self.data0.lines.close.get(ago=0, size=3))
        print("从-1开始往前取3天的收盘价:", self.data0.lines.close.get(ago=-1, size=3))
        print("从-2开始往前取3天的收盘价:", self.data0.lines.close.get(ago=-2, size=3))
        print("line的总长度:", self.data0.buflen())

        print(self.data0.lines.getlinealiases())

        self.sma1 = btind.SimpleMovingAverage(self.data)
        ema1 = btind.ExponentialMovingAverage(self.data)
        close_over_sma = self.data.close > self.sma1
        close_over_ema = self.data.close > ema1
        sma_ema_diff = self.sma1 - ema1
        self.buy_sig = bt.And(close_over_sma, close_over_ema, sma_ema_diff > 0)

    def log(self, txt, dt=None):
        '''可选，构建策略打印日志的函数：可用于打印订单记录或交易记录等'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已经处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log('BUY EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                        (order.ref, # 订单编号
                        order.executed.price, # 成交价
                        order.executed.value, # 成交额
                        order.executed.comm, # 佣金
                        order.executed.size, # 成交量
                        order.data._name)) # 股票名称
            else: # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                            (order.ref,
                            order.executed.price,
                            order.executed.value,
                            order.executed.comm,
                            order.executed.size,
                            order.data._name))
        pass

    def next(self):
        #print(f'datetime:{self.datas[0].datetime.date(0)}')
        #print(f'close:{self.data.close[0]},{self.data.close}')
        #print(self.data.close.get(ago = 0, size = 5))
        print('当前可用资金:', self.broker.getcash(), ' 当前总资产:', self.broker.getvalue(),' 当前持仓量:', self.broker.getposition(self.data).size,' 当前持仓成本:', self.broker.getposition(self.data).price,' 当前持仓量:', self.getposition(self.data).size,' 当前持仓成本:', self.getposition(self.data).price)
        if self.buy_sig:
            self.buy()
        pass

# 添加策略
cerebro.addstrategy(strategy)
# 添加策略分析指标
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # 回撤
# 启动回测
result = cerebro.run()
strat = result[0]
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())

# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())

cerebro.plot()


-------------self.datas-------------
-------------self.data-------------
AAPL <__main__.PandasData_more object at 0x000001E0A5E7FF10>
-------------self.data0-------------
AAPL <__main__.PandasData_more object at 0x000001E0A5E7FF10>
-------------self.datas[0]-------------
AAPL <__main__.PandasData_more object at 0x000001E0A5E7FF10>
0 索引： datetime 2022-11-23 close 151.070007
-1 索引： datetime 2022-11-22 close 150.179993
-2 索引 datetime 2022-11-21 close 148.009995
1 索引： datetime 2000-01-03 close 0.851942
2 索引 datetime 2000-01-04 close 0.780115
从 0 开始往前取3天的收盘价: array('d')
从-1开始往前取3天的收盘价: array('d', [151.289993, 148.009995, 150.179993])
从-2开始往前取3天的收盘价: array('d', [150.720001, 151.289993, 148.009995])
line的总长度: 5762
('close', 'low', 'high', 'open', 'volume', 'openinterest', 'datetime', 'pe', 'pb')
当前可用资金: 10000000.0  当前总资产: 10000000.0  当前持仓量: 0  当前持仓成本: 0.0  当前持仓量: 0  当前持仓成本: 0.0
当前可用资金: 10000000.0  当前总资产: 10000000.0  当前持仓量: 0  当前持仓成本: 0.0  当前持仓量: 0  当前持仓成本: 0.0
当前可用资金: 10000000.0  当前总资产: 10000

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]

In [ ]:
cerebro.